In [1524]:
pdf_path = "test/pdf/dc/form/Ceridian-2-EC-SDS-v2.1.pdf"
output_pdf_path = "test/target/dc/form/Ceridian-2-EC-SDS-v2.1.pdf"

# pdf_path = "test/pdf/dc/form/acuron-label.pdf"
# output_pdf_path = "test/target/dc/form/acuron-label.pdf"

# pdf_path = "test/pdf/dc/form/Blue_Book_2023_web.pdf"
# output_pdf_path = "test/target/dc/form/Blue_Book_2023_web.pdf"

In [1525]:
BOLD_FONTS_KEYWORDS = ["bold", "black", "heavy", "black", "extra", "ultra", "black"]

def is_bold_font(font):
    font = font.lower()
    for keyword in BOLD_FONTS_KEYWORDS:
        if keyword in font:
            return True
    return False

def is_same_font_family(font_a, font_b):
    return font_a.split("-")[0] == font_b.split("-")[0]

In [1526]:
def within_range(value, target, tolerance):
    return value >= target - tolerance and value <= target + tolerance

In [1527]:
import fitz
import json
import pandas as pd
import numpy as np
from pprint import pprint


In [1528]:
doc = fitz.open(pdf_path)

In [1529]:
doc.page_cropbox(0)

Rect(0.0, 0.0, 612.0, 792.0)

In [1530]:
doc.get_toc()

[]

In [1531]:
doc.page_count

4

In [1532]:
span_sizes = []
span_fonts = []

In [1533]:
for page in doc.pages():
    blocks = page.get_text("dict")["blocks"]

    for index, block in enumerate(blocks):
        for line in block.get("lines") or []:
            for span in line.get("spans") or []:
                span_sizes.append(span["size"])



In [1534]:
# Calculate the mean value of span sizes
mean_span_size = np.mean(span_sizes)
mean_span_size

8.530933659819624

In [1535]:
def is_block_text_format_consistent(block):
    lines = block.get("lines") or []
    spans = [span for line in lines for span in line.get("spans") or []]

    if len(spans) == 0:
        return False
    
    font = spans[0]["font"]
    size = spans[0]["size"]
    color = spans[0]["color"]
    origin = spans[0]["origin"]
    
    for span in spans:
        if span["font"] != font or span["size"] != size or span["color"] != color:
            return False
        
    return True

In [1536]:
def is_title(span, is_consistent):
    is_bold = is_bold_font(span["font"])

    if not is_consistent:
        print(span["text"], "not consistent")
        return False
    
    if not all(char.isalpha() or char.isspace() or char == ',' or char == '-' for char in span["text"]):
        print(span["text"], "not all characters are alpha, space, or comma")
        return False
    
    if span["size"] >= mean_span_size * 1.2:
        print(span["text"], "size >= mean_span_size * 1.5")
        return True
    
    if span["size"] >= mean_span_size * 0.8 and is_consistent and is_bold:
        print(span["text"], "size >= mean_span_size * 0.08 and is_consistent and is_bold")
        return True
    
    if span["text"] == span["text"].upper() and is_consistent and is_bold:
        print(span["text"], "text is all uppercase and is consistent")
        return True
    
    print(span["text"], "not title")
    return False
    

In [1537]:
for page_index, page in enumerate(doc):
    blocks = page.get_text("dict")["blocks"]

    for block_index, block in enumerate(blocks):
        if page_index == 12 and block_index == 14: 
            pprint(block)

        rect = fitz.Rect(block["bbox"])

        point = rect.tl
        point.x -= 10
        point.y += 8

        color = [1,0.7,0.7]
        color2 = [1,0.82,0.82]
        color3 = [0,0,1]

        page.draw_rect(
            rect,
            color=color,
            fill=None,
            width=1.2,
            overlay=True,
        )

        page.insert_text(point, f"{block_index}", color=[1,0,0], fontname="helvetica-bold", fontsize=8, overlay=True)


        is_consistent = is_block_text_format_consistent(block)

        lines = block.get("lines") or []        
        for line_index, line in enumerate(lines):
            rect = fitz.Rect(line["bbox"])

            page.draw_rect(
                rect,
                color=color2,
                fill=None,
                width=0.8,
                overlay=True,
            )

            spans = line.get("spans") or []
            for span_index, span in enumerate(spans):
                rect = fitz.Rect(span["bbox"])

                if is_title(span, is_consistent):
                    page.draw_rect(
                        rect,
                        color=color3,
                        fill=None,
                        width=1.2,
                        overlay=True,
                    )



Manufactured for: not consistent
Atticus, LLC not consistent
940 NW Cary Parkway, Suite 200   not consistent
Cary, NC 27513 not consistent
1 not all characters are alpha, space, or comma
1. IDENTIFICATION not all characters are alpha, space, or comma
PRODUCT NAME: not all characters are alpha, space, or comma
DESCRIPTION: not all characters are alpha, space, or comma
EPA REG. NO.: not all characters are alpha, space, or comma
COMPANY  size >= mean_span_size * 0.08 and is_consistent and is_bold
IDENTIFICATION: not all characters are alpha, space, or comma
Ceridian 2 EC not all characters are alpha, space, or comma
A liquid herbicide. not all characters are alpha, space, or comma
91234-154 not all characters are alpha, space, or comma
Atticus, LLC not consistent
940 NW Cary Parkway, Suite 200  not consistent
Cary, NC 27513 not consistent
2. HAZARD IDENTIFICATION not all characters are alpha, space, or comma
WARNING not consistent
May cause an allergic skin reaction  not consistent
(H317)

In [1538]:
doc.save(output_pdf_path)